In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import shapiro

In [ ]:
# load industry standard assessement CSV files
df_who = pd.read_csv('/Users/bk/Desktop/exist_centers/merged_data/who_merged.csv')
df_gad = pd.read_csv('/Users/bk/Desktop/exist_centers/merged_data/gad_merged.csv')
df_phq = pd.read_csv('/Users/bk/Desktop/exist_centers/merged_data/phq_merged.csv')
df_pcl = pd.read_csv('/Users/bk/Desktop/exist_centers/merged_data/ptsd_merged.csv')
df_ders = pd.read_csv('/Users/bk/Desktop/exist_centers/merged_data/ders_merged.csv')
df_ders2 = pd.read_csv('/Users/bk/Desktop/exist_centers/merged_data/ders2_merged.csv')

# load patient intake CSV file
df_intake = pd.read_csv('/Users/bk/Desktop/exist_centers/data/patient_intake_anon.csv')

In [ ]:
def clean_dates(col):
    return pd.to_datetime(col.astype(str).str.split(' ').str[0], format='mixed')

# who
df_who['assessment_date'] = clean_dates(df_who['assessment_date'])
df_who['score'] = df_who.iloc[:, -5:].sum(axis=1)
df_who.sort_values(['initial_group_identifier', 'assessment_date'], inplace=True)

# gad
df_gad['assessment_date'] = clean_dates(df_gad['assessment_date'])
# combining duplicate column
df_gad['5. * Being so restless that it is too hard to sit still'] = df_gad['5. * Being so restless that it is too hard to sit still'].combine_first(
  df_gad['5. * Being so restless that it’s hard to sit still'])
df_gad.drop('5. * Being so restless that it’s hard to sit still', inplace=True, axis=1)

df_gad['score'] = df_gad.iloc[:, -7:].sum(axis=1)
df_gad.sort_values(['initial_group_identifier', 'assessment_date'], inplace=True)

# phq
df_phq['assessment_date'] = clean_dates(df_phq['assessment_date'])
df_phq['score'] = df_phq.iloc[:, -9:].sum(axis=1)
df_phq.sort_values(['initial_group_identifier', 'assessment_date'], inplace=True)

# pcl (ptsd)
df_pcl['assessment_date'] = clean_dates(df_pcl['assessment_date'])
df_pcl['score'] = df_pcl.iloc[:, -20:].sum(axis=1)
df_pcl.sort_values(['initial_group_identifier', 'assessment_date'], inplace=True)

# ders
# some questions in DERS have reverse scoring
reverse = [1,2,6,7,8,10,17,20,22,24,34]
reversed_elements = [str(x) for x in reverse]
df_ders['assessment_date'] = clean_dates(df_ders['assessment_date'])
df_ders['score'] = df_ders.iloc[:, -36:].sum(axis=1)
df_ders.sort_values(['initial_group_identifier', 'assessment_date'], inplace=True)

# ders2
# reverese scored questions in DERS2 files need to be reformatted
reverse_cols2 = df_ders2.loc[:, df_ders2.columns.str.split('.').str[0].isin(reversed_elements)].columns
mapping = {"'-1":1,"'-2":2,"'-3":3,"'-4":4,"'-5":5}
df_ders2[reverse_cols2] = df_ders2[reverse_cols2].replace(mapping)
df_ders2['score'] = df_ders2.iloc[:, -36:].sum(axis=1)
df_ders2.sort_values(['initial_group_identifier', 'assessment_date'], inplace=True)

# Combining DERS files
df_ders2.columns=df_ders.columns
df_ders = pd.concat([df_ders,df_ders2])
df_ders.sort_values(['initial_group_identifier', 'assessment_date'], inplace=True)

# convert all date columns to datetime
for df in [df_who, df_gad, df_phq, df_pcl, df_ders]:
    df['assessment_date'] = pd.to_datetime(df['assessment_date'], errors='coerce')
    
# Combine all assessment data to one dataframe
cols = ['initial_group_identifier', 'assessment_date', 'score']
assessments = ['WHO', 'GAD', 'PHQ', 'PCL', 'DERS']

df_merge = df_who[cols].merge(df_gad[cols], how='outer', on=['initial_group_identifier', 'assessment_date'], suffixes=('_WHO', '_GAD'))
df_merge = df_merge.merge(df_phq[cols], how='outer', on=['initial_group_identifier', 'assessment_date'], suffixes=(None, '_PHQ'))
df_merge = df_merge.merge(df_pcl[cols], how='outer', on=['initial_group_identifier', 'assessment_date'], suffixes=(None, '_PCL'))
df_merge = df_merge.merge(df_ders[cols], how='outer', on=['initial_group_identifier', 'assessment_date'], suffixes=(None, '_DERS'))
df_merge.columns = ['initial_group_identifier', 'assessment_date'] + assessments

scores = df_merge.groupby('initial_group_identifier')[assessments].mean()

In [ ]:
scores

In [ ]:
scores.reset_index(inplace=True)

In [ ]:
scores.describe()

In [ ]:
scores.shape

## Merge Patient Intake Fields

In [ ]:
df_intake.columns

In [ ]:
drop_cols = ['group_identifier','mult_patient','admission_dt','discharge_dt','referrer']
sub_intake = df_intake.drop(columns=drop_cols)

sub_intake.head(10)

In [ ]:
df = scores.merge(sub_intake, on='initial_group_identifier')
df.head(10)

In [ ]:
df['status'].unique()

In [ ]:
df['discharge_type'].unique()

In [ ]:
df.describe()

## Merge Program Type

    - Located in clinical data report

    - requires a few series of merges:
        1. patient list and clinical data report
        2. then to current dataframe

In [ ]:
# load full patient list CSV
df_patient = pd.read_csv('/Users/bk/Desktop/exist_centers/data/patient_listv2.csv')

# load clinical data report CSV 
df_clinical = pd.read_csv('/Users/bk/Desktop/exist_centers/data/clinical_data_report_2025.csv')

In [ ]:
df_clinical.columns

In [ ]:
df_clinical.head(5)

In [ ]:
df_clinical['Program'].unique()

In [ ]:
df_patient

In [ ]:
len(df_patient['patient_ID'].unique())

In [ ]:
len(df_clinical['MR'].unique())

In [ ]:
sub_clinical = df_clinical[['First Name','MR','Program']]
sub_clinical = sub_clinical.rename(columns={'MR':'patient_ID'})

sub_clinical

In [ ]:
missing = df_patient[~df_patient['patient_ID'].isin(sub_clinical['patient_ID'])]

In [ ]:
missing

**Comment:** 
   - **Issue:** Newer/more recent patients in 2025 were not included in the clinical_data_report
   - **Issue Resolved:** Compiled a new clinical_data_report.csv in Kipu by changing the data range to include the patients from 2025

## MERGE 1

In [ ]:
df_program = df_patient.merge(sub_clinical, on='patient_ID')

In [ ]:
df_program

In [ ]:
df

## MERGE 2

In [ ]:
df = df.merge(df_program[['initial_group_identifier','Program']], on='initial_group_identifier')

In [ ]:
df.head(10)

In [ ]:
df[df['initial_group_identifier'].duplicated(keep=False)].sort_values(by='initial_group_identifier')

In [ ]:
df_program[df_program['first_name'].duplicated(keep=False)]

In [ ]:
sub1 = df[df['initial_group_identifier'].duplicated(keep=False)].sort_values(by='initial_group_identifier')
sub1['initial_group_identifier'].unique()